In [110]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### 1. Load and Visualize Data

Let's start by loading your entire dataset or a sample if it is a larger dataset.

In [111]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [112]:
### YOUR CODE HERE

# read dataset
# data_file_name = "./data/data.csv"
data_file_name = "/content/drive/My Drive/CSC542 - F3/data/data.csv"
data_file_handlle = pd.read_csv(data_file_name)

# define rows and coloums
fields_x = [
        'flow_duration', 'Header_Length', 'Protocol Type', 'Duration',
        'Rate', 'Srate', 'Drate', 'fin_flag_number', 'syn_flag_number',
        'rst_flag_number', 'psh_flag_number', 'ack_flag_number',
        'ece_flag_number', 'cwr_flag_number', 'ack_count',
        'syn_count', 'fin_count', 'urg_count', 'rst_count',
        'HTTP', 'HTTPS', 'DNS', 'Telnet', 'SMTP', 'SSH', 'IRC', 'TCP',
        'UDP', 'DHCP', 'ARP', 'ICMP', 'IPv', 'LLC', 'Tot sum', 'Min',
        'Max', 'AVG', 'Std', 'Tot size', 'IAT', 'Number', 'Magnitue',
        'Radius', 'Covariance', 'Variance', 'Weight',
]

fields_y = ['label']

In [113]:
# separate inputs and labels
data_x = data_file_handlle[fields_x]
data_y = data_file_handlle[fields_y]

In [114]:
### YOUR CODE HERE
def calcMaxMinMean(data):
    max_values = data.max(axis=0)
    min_values = data.min(axis=0)
    mean_values = data.mean(axis=0)

    return (max_values, min_values, mean_values)

In [115]:
# Precomputing values of min, max, mean and standard deviation for the columns
max_map = {}
min_map = {}
mean_map = {}
std_map = {}

for col_name in data_x.columns:
    column = data_x[col_name]

    max_map[col_name] = column.max()
    min_map[col_name] = column.min()
    mean_map[col_name] = column.mean()
    std_map[col_name] = column.std()

In [116]:
# Using Minimum and Maximum values of the columns to normalize the data

# As there are columns with same min and max values, leading to a NaN value for the normalization, we do not perform normalization on those columns

minmax_normalized_data = pd.DataFrame()

for col_name in data_x.columns:
    denominator = (max_map[col_name] - min_map[col_name])
    if denominator != 0:
        numerator = data_x[col_name] - min_map[col_name]
        minmax_normalized_data[col_name] = numerator / denominator


In [117]:
# Using Mean and Standard Deviation values of the columns to normalize the data

# As there are columns with zero standard deviation, leading to a NaN value for the normalization, we do not perform normalization on those columns

meanstd_normalized_data = pd.DataFrame()

for col_name in data_x.columns:
    denominator = std_map[col_name]
    if denominator != 0:
        numerator = data_x[col_name] - mean_map[col_name]
        meanstd_normalized_data[col_name] = numerator / denominator

Next, provide some characteristics about your output (e.g., histograms, spatial distribution of bounding boxes in the case of object detection, etc.). In the case of cases such as tasks such as reinforcement learning, you can provide details about the desired output and rewards used. You can show some correlations between inputs and outputs if possible.

In [118]:
# Using Minimum and Maximum values of the columns to normalize the data

# As there are columns with same min and max values, leading to a NaN value for the normalization, we do not perform normalization on those columns
minmax_normalized_data = pd.DataFrame()

for col_name in data_x.columns:
    denominator = (max_map[col_name] - min_map[col_name])
    if denominator != 0:
        numerator = data_x[col_name] - min_map[col_name]
        minmax_normalized_data[col_name] = numerator / denominator

In [119]:
# Using Mean and Standard Deviation values of the columns to normalize the data

# As there are columns with zero standard deviation, leading to a NaN value for the normalization, we do not perform normalization on those columns
meanstd_normalized_data = pd.DataFrame()

for col_name in data_x.columns:
    denominator = std_map[col_name]
    if denominator != 0:
        numerator = data_x[col_name] - mean_map[col_name]
        meanstd_normalized_data[col_name] = numerator / denominator

In [120]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
s_scaler = StandardScaler()

# scale the nd array
s_scaler.fit(data_x)
data_x_1 = s_scaler.transform(data_x)

# convert DF to NP array
X = data_x_1
Y = data_y.to_numpy()

In [121]:
# splitting data into training and validation sets
import random, math
def loadFeatures(X, Y, p, shift=0):
    # p is the percentage of data to be loaded into validation set

    total_data_entries = len(X)

    val_idx_list = list(np.round(np.linspace(0, total_data_entries - 1, math.floor(total_data_entries * (p/100)))).astype(int))
    if shift > 0:
        val_idx_list = [i+shift for i in val_idx_list if ((i+shift) < total_data_entries)]
    train_idx_list = list(set(np.array(range(total_data_entries))).difference(val_idx_list))

    # print(val_idx_list[:20])
    # print(train_idx_list[:20])

    return X[val_idx_list], X[train_idx_list], Y[val_idx_list], Y[train_idx_list]

In [122]:
xVal, xTrain, yVal, yTrain = loadFeatures(X, Y, 15, 1)

In [123]:
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay

# This function produces a summary of performance metrics including a confusion matrix
def summaryPerf(yTrain,yTrainHat,y,yHat,OH=False):
    # Plotting confusion matrix for the non-training set:
    # cm = metrics.confusion_matrix(y,yHat,normalize='true')
    # disp = metrics.ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=fields_y)
    # disp.plot()

    # Displaying metrics for training and non-training sets
    if OH:
        yTrain=np.argmax(yTrain,axis=1)
        y = np.argmax(y,axis=1)
    print('Training:  Acc = {:4.3f}'.format(metrics.accuracy_score(yTrain,yTrainHat)))
    print('Non-Train: Acc = {:4.3f}'.format(metrics.accuracy_score(y,yHat)))
    print('Training:  BalAcc = {:4.3f}'.format(metrics.balanced_accuracy_score(yTrain,yTrainHat,adjusted=True)))
    print('Non-Train: BalAcc = {:4.3f}'.format(metrics.balanced_accuracy_score(y,yHat)))

# RF todo

In [124]:
# from sklearn.ensemble import RandomForestClassifier
# from cuml import RandomForestClassifier as RandomForestClassifierGPU
# # Defining the model using default parameters
# M_RF = RandomForestClassifier()

# # Training the model
# M_RF.fit(xTrain,yTrain)

# MLP

In [125]:
import torch

# Setup device-agnostic code
if torch.cuda.is_available():
    device = "cuda" # NVIDIA GPU
elif torch.backends.mps.is_available():
    device = "mps" # Apple GPU
else:
    device = "cpu" # Defaults to CPU if NVIDIA GPU/Apple GPU aren't available

print(f"Using device: {device}")

Using device: cuda


In [126]:
all_attacks_list = ['DDoS-RSTFINFlood', 'DoS-TCP_Flood', 'DDoS-ICMP_Flood', 'DoS-UDP_Flood', 'DoS-SYN_Flood', 'Mirai-greeth_flood', 'DDoS-SynonymousIP_Flood', 'Mirai-udpplain', 'DDoS-SYN_Flood', 'DDoS-PSHACK_Flood', 'DDoS-TCP_Flood', 'DDoS-UDP_Flood', 'BenignTraffic', 'MITM-ArpSpoofing', 'DDoS-ACK_Fragmentation', 'Mirai-greip_flood', 'DoS-HTTP_Flood', 'DDoS-ICMP_Fragmentation', 'Recon-PortScan', 'DNS_Spoofing', 'DDoS-UDP_Fragmentation', 'Recon-OSScan', 'XSS', 'DDoS-HTTP_Flood', 'Recon-HostDiscovery', 'CommandInjection', 'VulnerabilityScan', 'DDoS-SlowLoris', 'Backdoor_Malware', 'BrowserHijacking', 'DictionaryBruteForce', 'SqlInjection', 'Recon-PingSweep', 'Uploading_Attack']

yLabels = sorted(all_attacks_list)
yLabelsDict = {}

for idx, l in enumerate(yLabels):
    yLabelsDict[l] = idx

yTrainInt = np.array([yLabelsDict[y[0]] for y in yTrain])
yValInt = np.array([yLabelsDict[y[0]] for y in yVal])

In [127]:
def createOH(arr, searchInDict=True):
    logit_len = len(all_attacks_list)

    OHArr = []

    for i in arr:
        tmp = [float(0.0)] * logit_len
        if searchInDict:
            tmp[all_attacks_list.index(i[0])] = float(1.0)
        else:
            tmp[i] = float(1.0)
        tmp = np.array(tmp)
        OHArr.append(tmp)

    return np.array(OHArr)


In [128]:
logit_len = len(all_attacks_list)

yTrainOH = createOH(yTrain, True)
yValOH = createOH(yVal, True)

In [129]:
import torch.nn as nn
import torch.nn.init as init

# Define the model architecture
class MLP(nn.Module):
    def __init__(self,device,epochs,weights):
        super(MLP, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(46, 100),
            nn.ReLU(),
            nn.Linear(100, 64),
            nn.Dropout(0.05),
            nn.ReLU(),
            nn.Linear(64, 30),
            nn.Dropout(0.10),
            nn.ReLU(),
            nn.Linear(30, 34),
        ).to(device)

        for layer in self.model:
            if isinstance(layer, nn.Linear):
                init.xavier_normal_(layer.weight)

        self.device = device
        self.epochs = epochs
        self.loss_fn = nn.CrossEntropyLoss(weight = torch.tensor(weights))
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=2.5e-3)

    def forward(self, x):
        logits = self.model(x)
        return logits

    def fit(self,X,y):
        X = torch.from_numpy(X).float()
        X = X.to(device)
        y = torch.from_numpy(y).float()
        y = y.to(device)

        for t in range(self.epochs):
            pred = self.model(X)
            loss = self.loss_fn(pred, y)

            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

            if t%100 == 99:
                print("[Epoch {t:5d} of {epochs}] loss: {loss:1.6f}".format(
                    t=t+1,epochs=self.epochs,loss=loss))

    def predict(self,X):
        X = torch.from_numpy(X).float()
        X = X.to(self.device)

        pred = self.model(X)
        pred = pred.cpu().detach().numpy()
        pred = np.argmax(pred,axis=1)

        return pred

In [130]:
#calc weights and set epoch

unique, counts = np.unique(yTrain, return_counts=True)
counts = dict(zip(unique, counts))
total_count = yTrain.size
weights_hp = []
for i in counts.values():
    tmp = 1 - (i / total_count)
    weights_hp.append(float(tmp))

In [131]:
epochs_hp = 2000

In [132]:
MLPModel = MLP(device,epochs_hp,weights_hp).to(device)
print(MLPModel.model)
MLPModel.fit(xTrain, yTrainOH)

Sequential(
  (0): Linear(in_features=46, out_features=100, bias=True)
  (1): ReLU()
  (2): Linear(in_features=100, out_features=64, bias=True)
  (3): Dropout(p=0.05, inplace=False)
  (4): ReLU()
  (5): Linear(in_features=64, out_features=30, bias=True)
  (6): Dropout(p=0.1, inplace=False)
  (7): ReLU()
  (8): Linear(in_features=30, out_features=34, bias=True)
)
[Epoch   100 of 2000] loss: 0.491710
[Epoch   200 of 2000] loss: 0.437271
[Epoch   300 of 2000] loss: 0.406753
[Epoch   400 of 2000] loss: 0.373629
[Epoch   500 of 2000] loss: 0.319703
[Epoch   600 of 2000] loss: 0.275308
[Epoch   700 of 2000] loss: 0.279189
[Epoch   800 of 2000] loss: 0.195223
[Epoch   900 of 2000] loss: 0.147429
[Epoch  1000 of 2000] loss: 0.233739
[Epoch  1100 of 2000] loss: 0.118073
[Epoch  1200 of 2000] loss: 0.098481
[Epoch  1300 of 2000] loss: 0.085737
[Epoch  1400 of 2000] loss: 0.078899
[Epoch  1500 of 2000] loss: 0.073883
[Epoch  1600 of 2000] loss: 0.069364
[Epoch  1700 of 2000] loss: 0.067056
[Epoch

In [133]:
yTrainHat = MLPModel.predict(xTrain)
yValHat = MLPModel.predict(xVal)

yTrainHatOH = createOH(yTrainHat, False)
yValHatOH = createOH(yValHat, False)

In [134]:
print('Results for Validation:\n')
summaryPerf(yTrainOH,yTrainHat,yValOH,yValHat,True)

Results for Validation:

Training:  Acc = 0.982
Non-Train: Acc = 0.982
Training:  BalAcc = 0.620
Non-Train: BalAcc = 0.626


# LG

In [ ]:
from sklearn.linear_model import LogisticRegression

model_lg = LogisticRegression(n_jobs=-1)

model_lg.fit(xTrain, yTrain.ravel())

In [ ]:
yTrainHat1 = model_lg.predict(xTrain)
yValHat1 = model_lg.predict(xVal)

print('Results for Validation:\n')
summaryPerf(yTrainOH,yTrainHat1,yValOH,yValHat1)